In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import ksone
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
import re
from joblib import dump, load
import joblib  

In [2]:
df = pd.read_csv('diamonds.csv')
df_coords = pd.read_csv('coords_diamonds.csv')
df = df.drop('Unnamed: 0', axis=1)

## Visualización

In [3]:
# for x in df.columns:
#     print(x)
#     # Configurar el gráfico
#     plt.figure(figsize=(10, 6))
#     sns.lineplot(data=df[x], palette='BuPu')  # Seaborn usará el índice para el eje X



#     # Mostrar el gráfico
#     plt.show()

## Homologación

CUT

In [4]:
# Función para limpiar los valores
def limpiar_valor(valor):
    # Eliminar caracteres especiales
    valor_limpio = re.sub(r"[^a-zA-Z\s]", '', valor)
    # Corregir errores comunes y homologar
    correcciones = {
        'Very Goo': 'Very Good',
        'Very Go': 'Very Good',
        'V&ery Good': 'Very Good',
        'Very G#ood': 'Very Good',
        "Very *'Good": 'Very Good',
        'Very Go#od': 'Very Good',
        'Ide&al': 'Ideal',
        'Ide!al': 'Ideal',
        'Id!eal': 'Ideal',
        "Ide*'al": 'Ideal',
        '*Ideal': 'Ideal',
        'I#deal': 'Ideal',
        '&Ideal': 'Ideal',
        'Pre!mium': 'Premium',
        'Pr?emium': 'Premium',
        "P*'remium": 'Premium',
        'P?remium': 'Premium',
        '&Premium': 'Premium',
        'Go?od': 'Good',
        'G#ood': 'Good',
        '!Good': 'Good'
    }
    return correcciones.get(valor_limpio, valor_limpio)

# Limpiar y homologar los valores de la columna
df['cut'] = df['cut'].apply(limpiar_valor)

COLOR

In [5]:
df['color'] = df['color'].replace(to_replace=r"[^EIJHFGD]", value='', regex=True)

CLARITY

In [6]:
mapeo = {
    'SI2': 'SI2', 'SI1': 'SI1', 'VS1': 'VS1', 'VS2': 'VS2', 'VVS2': 'VVS2', 'VVS1': 'VVS1',
    'I1': 'I1', 'IF': 'IF',
    'S?I1': 'SI1', 'SI!1': 'SI1', '&VS2': 'VS2', '&SI2': 'SI2', "S*'I1": 'SI1',
    'VS?1': 'VS1', "S*'I2": 'SI2', '#VS1': 'VS1', 'V&S2': 'VS2', 'V!S2': 'VS2',
    '!VS2': 'VS2', 'VS#2': 'VS2', "VVS*'2": 'VVS2', "*'SI2": 'SI2', 'VV?S1': 'VVS1',
    'S&I1': 'SI1', "*'SI1": 'SI1', 'SI?1': 'SI1', 'VV#S1': 'VVS1', 'V#S2': 'VS2',
    '#SI!1': 'SI1', 'S!I2': 'SI2'
}



# Aplicar la función replace con regex=True para habilitar expresiones regulares
df['clarity'] = df['clarity'].replace(mapeo, regex=False)


## Validación

columnas númericas

In [7]:
num_columns = df.select_dtypes(include=['int64', 'float64']).columns
def validate_num_columns(df, num_columns):
    for column in num_columns:
        tipo_dato = df[column].dtypes

        # Verificar si el tipo de dato es numérico
        if tipo_dato == 'int' or tipo_dato == 'float':
            print("La columna es completamente numérica")
        else:
            print(f"La columna {column} tiene datos no numéricos")
validate_num_columns(df,num_columns)

La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica


columnas de texto

In [8]:
num_columns = df.select_dtypes(include=['object']).columns
def categorical_column(df, num_columns):
    for column in num_columns:
        tipo_dato = df[column].dtypes

        # Verificar si el tipo de dato es numérico
        if tipo_dato == 'object' or tipo_dato == 'object':
            print("La columna es completamente texto")
        else:
            print(f"La columna {column} tiene datos no texto")

categorical_column(df, num_columns)

La columna es completamente texto
La columna es completamente texto
La columna es completamente texto


## Missings values

Creamos nuestros conjuntos de entrenamiento y validación

In [9]:
X_train,X_test=train_test_split(df,test_size=.2,random_state=413)

In [10]:
def missing_values(df,num_columns):
    for column in num_columns:
        mean = df[column].dropna().mean()
        median = df[column].dropna().median()
        mode = df[column].dropna().mode()[0]  # mode() devuelve una Serie, toma el primer valor
        
        print(f"\n{column}: La media es {mean}, la mediana es {median}, y el modo es {mode}")
        
        for metric in [mean, median, mode]:
            originales = list(df[column].dropna().values)
            imputados = list(df[column].fillna(metric).values)
            
            # Ejecutar la prueba KS
            ks_stat, ks_pvalue = stats.ks_2samp(originales, imputados)
            
            # Imprimir o almacenar los resultados
            print(f"Prueba KS para {metric}: estadístico={ks_stat}, p-valor={ks_pvalue}")

In [11]:
num_columns = df.columns[df.isna().any()].tolist()
missing_values(X_train,num_columns)


depth: La media es 61.7516227951819, la mediana es 61.8, y el modo es 62.0
Prueba KS para 61.7516227951819: estadístico=0.029593963871762496, p-valor=2.2130365911915135e-16
Prueba KS para 61.8: estadístico=0.02748194668708115, p-valor=3.4751680220647744e-14
Prueba KS para 62.0: estadístico=0.029883689306071326, p-valor=1.0741422552910769e-16

x: La media es 5.728688699108136, la mediana es 5.69, y el modo es 4.37
Prueba KS para 5.728688699108136: estadístico=0.01039515926092327, p-valor=0.01967489540692731
Prueba KS para 5.69: estadístico=0.010113691511548506, p-valor=0.025180008502440954
Prueba KS para 4.37: estadístico=0.01789234955124283, p-valor=2.2814308079606564e-06

y: La media es 5.731230639454507, la mediana es 5.71, y el modo es 4.33
Prueba KS para 5.731230639454507: estadístico=0.005548181400352514, p-valor=0.5218516493109235
Prueba KS para 5.71: estadístico=0.005405387063054268, p-valor=0.555752257300214
Prueba KS para 4.33: estadístico=0.009979117586142616, p-valor=0.0276

Mejor valor a imputar:
- Para "depth", se recomienda la mediana.
- Para "x", se recomienda la mediana, aunque la diferencia con la media es mínima.
- Para "y", se recomienda la mediana, dada su semejanza más cercana con la distribución original.

In [12]:
def complete_missing_values(imp,num_columns):
    for column in num_columns:
        X_train[column]=imp.fit_transform(X_train[[column]])
        X_test[column]=imp.transform(X_test[[column]])

imp=SimpleImputer(missing_values=np.nan,strategy="median")
complete_missing_values(imp,num_columns)

/tmp/ipykernel_2449036/3201589919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[column]=imp.fit_transform(X_train[[column]])
/tmp/ipykernel_2449036/3201589919.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[column]=imp.transform(X_test[[column]])
/tmp/ipykernel_2449036/3201589919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

## Outliers

Unificamos en df limpio

In [13]:
df = pd.concat([X_train, X_test], axis=0)

- n_estimators: número de árboles que forman el modelo.

- max_samples: número de observaciones empleadas para entrenar cada árbol.

- max_features : El número de características que se extraerán para entrenar cada estimador base.

- contamination: proporción de anomalías esperadas en los datos de entrenamiento. En base a este valor, se establece el límite acorde al cual se clasifican las observaciones en normales o anómalas.

- random_state: semilla para garantizar la reproducibilidad de los resultados.

Se procede a entrenar un modelo asumiendo que hay un 1% de observaciones anómalas en el conjunto de entrenamiento.


In [14]:
df_iso = df.select_dtypes(include=['int64', 'float64'])
max_features=df_iso.shape[1]
n_estimators=50
max_samples='auto'
contamination=float(0.06)
model=IsolationForest(max_features = max_features, n_estimators=n_estimators, max_samples=max_samples, contamination=contamination)
model.fit(df_iso)
df_iso["outlier"]=model.predict(df_iso)
df = pd.concat([df_iso, df[['cut', 'color', 'clarity']]], axis=1)
df = df[df["outlier"]!=-1].reset_index(drop=True)
df = df.drop('outlier', axis=1)

/tmp/ipykernel_2449036/779421519.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iso["outlier"]=model.predict(df_iso)


In [15]:
df_iso.outlier.value_counts()

 1    50694
-1     3236
Name: outlier, dtype: int64

In [16]:
df_iso.outlier.value_counts()

 1    50694
-1     3236
Name: outlier, dtype: int64

In [17]:
df_iso.outlier.value_counts()

 1    50694
-1     3236
Name: outlier, dtype: int64

In [18]:
# for x in df.columns:
#     print(x)
#     # Configurar el gráfico
#     plt.figure(figsize=(10, 6))
#     sns.lineplot(data=df[x], palette='BuPu')  # Seaborn usará el índice para el eje X



#     # Mostrar el gráfico
#     plt.show()

## Data Engineering

In [19]:
num_columns = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df,columns = num_columns, prefix_sep='_') 

## Correlación

In [20]:
def correlation(df):
    corr=df.corr(method="spearman")
    corr=abs(corr)
    corr=corr[['price']].sort_values(by = 'price' , ascending = False)
    list_corr = list(corr[corr['price']>.01].index)
    df = df[list_corr]
    return df, list_corr

df, list_corr = correlation(df)


## Normalización

In [21]:
precio_scaler = MinMaxScaler()
precio_scaler.fit(df[['price']])
joblib.dump(precio_scaler, 'price_scaler.joblib')

['price_scaler.joblib']

In [22]:
def normalize(df):
    X_train,X_test=train_test_split(df,test_size=.2,random_state=413)
    scaler = MinMaxScaler()
    scaler.fit(X_train)

    index_train = X_train.index
    index_test = X_test.index

    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=index_test)
    X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=index_train)
    df = pd.concat([X_train, X_test], axis=0)
    return df

df = normalize(df)

## Modelos

### Regresión Lineal

In [32]:
# Definir las características y la variable objetivo
X = df.drop('price', axis=1)
y = df['price']

In [33]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Inicializar y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)


# Predecir en el conjunto de entrenamiento y en el conjunto de prueba
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


# Evaluar el modelo para el conjunto de entrenamiento y prueba
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

In [39]:
print(f"Train : MSE = {mse_train:.6f}, R^2 = {r2_train:.2f}")
print(f"Test: MSE = {mse_test:.6f}, R^2 = {r2_test:.2f}")

Train : MSE = 0.002812, R^2 = 0.91
Test: MSE = 0.002718, R^2 = 0.91


 El modelo no está sobreajustado de manera significativa, ya que no hay una gran diferencia entre el MSE del conjunto de entrenamiento y el de prueba, lo que sugiere que el modelo tiene un rendimiento consistente en ambos conjuntos.

 Los resultados del Score es un indicativo de un buen ajuste y de una buena capacidad de generalización del modelo. Lo que significa que el modelo puede explicar el 90% de la variabilidad en los precios de los diamantes, lo cual son excelentes resultados

### SVM

In [35]:
# Importar las librerías necesarias
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score


# Inicializar y entrenar el modelo SVR
svr = SVR(kernel='rbf')
svr.fit(X_train, y_train)

# Predecir en el conjunto de entrenamiento y en el conjunto de prueba
y_train_pred = svr.predict(X_train)
y_test_pred = svr.predict(X_test)
# Evaluar el modelo para el conjunto de entrenamiento
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Evaluar el modelo para el conjunto de prueba
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Imprimir los resultados
print(f"Rendimiento en el conjunto de entrenamiento: MSE = {mse_train:.6f}, R^2 = {r2_train:.2f}")
print(f"Rendimiento en el conjunto de prueba: MSE = {mse_test:.6f}, R^2 = {r2_test:.2f}")


Rendimiento en el conjunto de entrenamiento: MSE = 0.002129, R^2 = 0.93
Rendimiento en el conjunto de prueba: MSE = 0.002139, R^2 = 0.93


Tanto el modelo de regresión lineal como el modelo de Máquinas de Soporte Vectorial (SVM) han demostrado ajustarse bastante bien a nuestros datos.
Podemos concluir que estamos bien posicionados para proporcionar estimaciones de precios de diamantes confiables y precisas

# How much are stolen diamonds worth?

Guardamos la versión final del modelo que usaremos para nuestras predicciones

In [26]:
stolendiamonds = pd.read_csv('StolenDiamonds.csv')

In [27]:
# from joblib import dump, load

# # Guardar el modelo a un archivo
# dump(model, 'model_regresion_lineal.joblib')

In [28]:
def preprocessing(df, list_corr, num_columns_numeric, num_columns_categoric):
    df['cut'] = df['cut'].apply(limpiar_valor)
    df['color'] = df['color'].replace(to_replace=r"[^EIJHFGD]", value='', regex=True)
    df['clarity'] = df['clarity'].replace(mapeo, regex=False)
    validate_num_columns(df, num_columns_numeric)
    categorical_column(df, num_columns_categoric)
    df = pd.get_dummies(df, columns=num_columns_categoric, prefix_sep='_')  
    
    for column in list_corr:
        if column not in df.columns:
            df[column] = 0  # Crea la columna y asigna el valor '0'
    
    df = df[list_corr]  # Se asegura de que df solo contenga las columnas en list_corr
    df = normalize(df)
    return df

In [29]:
stolendiamonds.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [30]:
num_columns_numeric = stolendiamonds.select_dtypes(include=['int64', 'float64']).columns
num_columns_categoric = stolendiamonds.select_dtypes(include=['object']).columns
list_corr = ['carat', 'y', 'z', 'x', 'table', 'clarity_SI2', 'clarity_VVS1',
 'cut_Ideal', 'cut_Premium', 'color_E', 'clarity_IF', 'clarity_VVS2', 'color_J',
 'color_D', 'cut_Fair', 'color_H', 'clarity_SI1', 'color_I', 'depth',
 'cut_Good', 'clarity_I1', 'cut_Very Good', 'clarity_VS1', 'color_G']
df_preprocessed = preprocessing(stolendiamonds, list_corr, num_columns_numeric, num_columns_categoric)

La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente numérica
La columna es completamente texto
La columna es completamente texto
La columna es completamente texto


In [31]:
model_loaded = load('model_regresion_lineal.joblib')
precio_scaler_loaded = joblib.load('price_scaler.joblib')

# Hacer predicciones con el modelo
y_pred = model_loaded.predict(df_preprocessed)

# Realizar la transformación inversa de las predicciones de 'precio'
y_pred_inverse = precio_scaler_loaded.inverse_transform(y_pred.reshape(-1, 1))

## Price of stolen diamonds

In [33]:
y_pred_inverse

array([[11454.21761658],
       [ 4686.66986356],
       [13085.49842229],
       [  945.6321092 ],
       [ 4118.38291106],
       [ 9870.29362177],
       [10215.38293269],
       [ 3358.51625878],
       [15260.520776  ],
       [10835.30156777]])